# Lab 3: Mixtral-8x7B Performance Optimization Challenge

## Objective
Optimize the training performance of a **Mixtral-8x7B** Mixture of Experts (MoE) model to achieve the highest possible **Model FLOPs Utilization (MFU)**.

## Workshop Structure
1. **Baseline profiling**: Profile an unoptimized training run
2. **Identify bottlenecks**: Use profiling tools to find inefficiencies
3. **Apply optimizations**: Implement kernel and parallelism optimizations
4. **Measure improvement**: Quantify MFU and throughput gains

## Topics Covered
- Distributed training parallelism (FSDP, EP, PP)
- Performance profiling with torch.profiler and Nsight
- Understanding arithmetic intensity
- Developing strategic optimization plans

## Grading (35 points)

| Component | Points | Description |
|-----------|--------|-------------|
| Profiling Analysis | 5 | Try profile with nsys file |
| MFU Improvement | 35 | Points based on achieved MFU (see rubric below) |

### MFU Scoring Rubric (35 points)
| MFU Achieved | Points |
|--------------|--------|
| ≥ 20.5% | 35 (full marks) |
| 1% - 20.5% | Linear: (MFU - 1) / 19.5 × 35 |
| < 1% | 0 |

## Hardware Requirements
- 8x NVIDIA H100 GPUs (or equivalent)
- Runtime container: `nvcr.io/nvidia/nemo-automodel:25.11`

## Submission
Submit the following files in your `submission/` folder:
1. **Nsight profile** (`automodel_profile_*.nsys-rep`) — from Part 2
2. **Optimized recipe** (`optimized_mixtral-8x7b-v0-1_benchmark.yaml`) — your configuration changes
3. **Benchmark results** (`benchmark_results.json`) — auto-generated MFU output

See **Part 5: Submission Checklist** for detailed instructions.

## Environment Setup

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/NVIDIA-NeMo/Automodel.git
!mkdir -p submission
%cd Automodel
!git fetch origin zhiyul/llm-optimization-workshop
!git checkout zhiyul/llm-optimization-workshop

In [ ]:
%%writefile ../submission/optimized_mixtral-8x7b-v0-1_benchmark.yaml
# Copyright (c) 2025, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# torchrun --nproc-per-node=8 --nnodes=1 nemo_automodel/recipes/llm/benchmark.py --config examples/llm_finetune/mistral/mixtral-8x7b-v0-1_benchmark.yaml
seed: 42

# Benchmark section
benchmark:
  warmup_steps: 5   
  peak_tflops: 989  # H100: 989, A100: 312
  nsys_start: -1    # Set to step number to profile (e.g., 10)
  nsys_end: -1      # Set to end step (e.g., 15)
  nsys_ranks: []    # e.g., [0] to profile rank 0
  num_nodes: 1

step_scheduler:
  global_batch_size: 256
  local_batch_size: 1   # open for change
  ckpt_every_steps: 50
  val_every_steps: 1000
  max_steps: 10

dist_env:
  backend: nccl
  timeout_minutes: 1

rng:
  _target_: nemo_automodel.components.training.rng.StatefulRNG
  seed: 1111
  ranked: true

model:
  _target_: nemo_automodel.components.models.mixtral.model.MixtralForCausalLM.from_pretrained
  pretrained_model_name_or_path: mistralai/Mixtral-8x7B-v0.1
  torch_dtype: bf16
  trust_remote_code: true
  router_aux_loss_coef: 0.0  # Disable aux/load-balancing loss

checkpoint:
  enabled: true
  checkpoint_dir: checkpoints/
  model_save_format: torch_save # torch_save or safetensors
  save_consolidated: false
  load_base_model: false

distributed:
  _target_: nemo_automodel.components.distributed.fsdp2.FSDP2Manager
  dp_size: None
  tp_size: 8
  cp_size: 1
  use_hf_tp_plan: true

loss_fn:
  _target_: nemo_automodel.components.loss.masked_ce.MaskedCrossEntropy

# Use MockIterableDataset for benchmarking (faster, no I/O)
dataset:
  _target_: nemo_automodel.components.datasets.llm.mock_iterable_dataset.MockIterableDataset
  vocab_size: 100
  seq_len: 1024
  num_samples: 1000000

dataloader:
  _target_: torch.utils.data.DataLoader
  batch_size: null  # Dataset already yields batches
  # Note: model_config will be auto-injected by train_ft.py for PP models

optimizer:
  _target_: torch.optim.Adam
  betas: [0.9, 0.999]
  eps: 1e-8
  lr: 1.0e-5
  weight_decay: 0

lr_scheduler:
  lr_decay_style: cosine
  min_lr: 1.0e-6 

In [ ]:
# Validate your config - run this before submitting!
import yaml
from typing import Any

# Fields that MUST remain unchanged
FROZEN_FIELDS = {
    "seed": 42,
    "benchmark.warmup_steps": 5,
    "benchmark.peak_tflops": 989,
    "benchmark.num_nodes": 1,
    "step_scheduler.global_batch_size": 256,
    "step_scheduler.max_steps": 10,
    "model._target_": "nemo_automodel.components.models.mixtral.model.MixtralForCausalLM.from_pretrained",
    "model.pretrained_model_name_or_path": "mistralai/Mixtral-8x7B-v0.1",
    "model.torch_dtype": "bf16",
    "dataset._target_": "nemo_automodel.components.datasets.llm.mock_iterable_dataset.MockIterableDataset",
    "dataset.seq_len": 1024,
}

def get_nested_value(config: dict, key_path: str) -> Any:
    """Get nested value from config using dot notation."""
    keys = key_path.split(".")
    value = config
    for key in keys:
        if isinstance(value, dict) and key in value:
            value = value[key]
        else:
            return None
    return value

def validate_config(config_path: str) -> bool:
    """Validate that the config only changes allowed fields."""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    
    valid = True
    print("🔍 Validating config...")
    print("-" * 50)
    
    # Check frozen fields
    for field, expected in FROZEN_FIELDS.items():
        actual = get_nested_value(config, field)
        if actual != expected:
            print(f"❌ FROZEN field '{field}' was changed!")
            print(f"   Expected: {expected}, Got: {actual}")
            valid = False
    
    # Show distributed config
    dist = config.get("distributed", {})
    print(f"\n📊 Parallelism config:")
    print(f"   TP={dist.get('tp_size', 'None')}, "
          f"PP={dist.get('pp_size', 'None')}, "
          f"EP={dist.get('ep_size', 'None')}, "
          f"CP={dist.get('cp_size', 'None')}")
    print(f"   local_batch_size={config.get('step_scheduler', {}).get('local_batch_size', 1)}")
    
    print("-" * 50)
    if valid:
        print("✅ Config is valid! Ready to benchmark.")
    else:
        print("❌ Config validation FAILED. Please fix the errors above.")
    
    return valid

# Run validation
validate_config("../submission/optimized_mixtral-8x7b-v0-1_benchmark.yaml")

---
## Part 1: Run baseline

> **Tip**: Running in a **terminal** is recommended for real-time output. Notebook cells also work, but may incur additional memory overhead.

The benchmark script will output:
- Per-iteration timing and loss
- Average MFU (Model FLOPs Utilization)
- JSON summary saved to `../submission/benchmark_results.json` (configured in YAML)

In [ ]:
%%bash
# Note: This is very slow with ~1% MFU - just run 2 steps for verification
torchrun --nproc-per-node=8 --nnodes=1 \
    nemo_automodel/recipes/llm/benchmark.py \
    --config ../submission/optimized_mixtral-8x7b-v0-1_benchmark.yaml \
    --benchmark.warmup_steps=1 \
    --step_scheduler.max_steps=2

---
## Part 2: Profiling Analysis (5 points)

Use profiling tools to identify performance bottlenecks.

### Profiling Tools Available
1. **Nsight Systems**: GPU kernel analysis, timeline visualization
2. **torch.profiler**: PyTorch operation profiling
3. **CUDA Events**: Fine-grained kernel timing

We choose to use Nsight System inside the lab.

### Enable Nsight Profiling

Modify the benchmark config to enable Nsight profiling:

```yaml
benchmark:
  nsys_start: 6      # Start profiling at step 6 (after warmup)
  nsys_end: 6        # End profiling at step 6
  nsys_ranks: [0]    # Profile rank 0
```

### What to Look For
Open the `.nsys-rep` file in Nsight Systems GUI for analysis (from https://developer.nvidia.com/nsight-systems/get-started) and find bottlenecks.

Run with the command line as follows:

In [ ]:
%%bash
mkdir -p ../profile
TIMESTAMP=$(date +"%Y%m%d_%H%M%S")
PROFILE_FILE="../profile/automodel_profile_${TIMESTAMP}.nsys-rep"

nsys profile -s none \
    --trace=nvtx,cuda \
    --cudabacktrace=all \
    --cuda-graph-trace=node \
    --python-backtrace=cuda \
    --wait all \
    -o ${PROFILE_FILE} \
    --force-overwrite true \
    --capture-range=cudaProfilerApi \
    --capture-range-end=stop \
    torchrun --nproc-per-node=8 --nnodes=1 \
    nemo_automodel/recipes/llm/benchmark.py \
    --config ../submission/optimized_mixtral-8x7b-v0-1_benchmark.yaml \
    --benchmark.nsys_start=6 --benchmark.nsys_end=6 --benchmark.nsys_ranks=[0]

---
## Part 3: Optimization Plan (35 points)

Based on your profiling analysis, develop an optimization strategy and update the recipe ../submission/optimized_mixtral-8x7b-v0-1_benchmark.yaml

### Hints for Available Optimization Techniques

#### 1. Distributed Parallelism
| Technique | Description | Config Key |
|-----------|-------------|------------|
| Tensor Parallel (TP) | Split model tensors across GPUs | `distributed.tp_size` |
| Expert Parallel (EP) | Distribute MoE experts across GPUs | `distributed.ep_size` |
| Pipeline Parallel (PP) | Split model layers across GPUs | `distributed.pp_size` |
| Data Parallel (FSDP) | Replicate model, split data | `distributed.dp_size` |
| Context Parallel (CP) | Split sequence across GPUs | `distributed.cp_size` |

#### 2. Batch Optimization
- **Local batch size**: Increase GPU utilization

#### 3. Memory Optimizations
- **activation_checkpointing**: Trade compute for memory

#### 4. Kernel
Go throught the code to see if there is any kernel option to try.

---
## Part 4: Implement Optimizations (35 points)

Create/update your optimized configuration and run the benchmark.

In [ ]:
%%bash
torchrun --nproc-per-node=8 --nnodes=1 \
    nemo_automodel/recipes/llm/benchmark.py \
    --config ../submission/optimized_mixtral-8x7b-v0-1_benchmark.yaml \
    --benchmark.json_output_path=../submission/benchmark_results.json

---
## Part 5: Submission Checklist

Before submitting, ensure your `submission/` folder contains all required files.

### Required Files

| File | Points | Description |
|------|--------|-------------|
| `automodel_profile_*.nsys-rep` | 5 | Nsight Systems profile from Part 2 |
| `optimized_mixtral-8x7b-v0-1_benchmark.yaml` | 35 | Your optimized recipe configuration |
| `benchmark_results.json` | — | Benchmark results with MFU (auto-generated) |

### Step-by-Step Submission Guide

**Step 1: Copy your Nsight profile to the submission folder**
```bash
cp ../profile/automodel_profile_*.nsys-rep ../submission/
```

**Step 2: Verify your optimized recipe**
> The recipe is already saved via the `%%writefile` cell in Environment Setup.  
> Re-run that cell if you made changes to your configuration.

**Step 3: Run the final benchmark to generate results**
```bash
torchrun --nproc-per-node=8 --nnodes=1 \
    nemo_automodel/recipes/llm/benchmark.py \
    --config ../submission/optimized_mixtral-8x7b-v0-1_benchmark.yaml
    --benchmark.json_output_path=../submission/benchmark_results.json
```

### Verify Your Submission

```bash
ls -la ../submission/
```

**Expected output:**
```
submission/
├── optimized_mixtral-8x7b-v0-1_benchmark.yaml  # Your optimized config
├── benchmark_results.json                       # Benchmark output with MFU score
└── automodel_profile_*.nsys-rep                 # Nsight profile for grading
```

### Grading Summary

| Component | Points | Criteria |
|-----------|--------|----------|
| Nsight Profile | 5 | Valid `.nsys-rep` file submitted |
| MFU Score | 35 | Based on `benchmark_results.json` (see rubric in intro) |
| **Total** | **40** | |